In [8]:
#Note that this only works in a polynomial Ring P (Like def. below)
def FactorBasis(m,q):
    R.<x> = GF(q,'a')
    P = PolynomialRing(R, 'y')
    y = P.gen()
    B = []
    for p in P.monics(max_degree=m):
        if p.is_irreducible():
            B.append(p)
    return B

def Division(t,B,q):
    R.<x> = GF(q,'a')
    P = PolynomialRing(R,'y')
    y = P.gen()
    C = [0]*(len(B))
    for i in range(0,len(B)):
        while (B[i].divides(t) == True):
            C[i]+=1
            t = t.quo_rem(B[i])[0]
    remainder = t
    return C,remainder
#The following definition returns (g mod n)^y mod n
R.<x> = GF(101,'a')
P = PolynomialRing(R,'y')
y = P.gen()
def d(f,h,k):
    s = f%h
    v = s^k
    t = v%h
    return t

In [9]:
q = 101
R.<x> = GF(q,'a')
P = PolynomialRing(R, 'y')
y = P.gen()
    
def get_order(g, n, q, m, rels=10, verbose=False, alphas=False, texify=True):
       
    # INPUT:
    # q = in which F_q field do we want to work?
    # m = maximal order of irreducible polynomial in Factor Base
    # n = modulus (So module which polynomial)
    # g = base (So for which polynomial do we want to find the order?)
    # B = bound for primes NOTE: THIS WILL BE LEFT OUT
    # rels = number of additional relations
    # verbose = whether to print info as you go
    # alphas = whether to return the alpha values
    # textify = whether to print latex for algorithm
    # OUTPUT:  order of g modulo n (with high probability)
    if GCD(g,n) == 1:
        R.<x> = GF(q,'a')
        P = PolynomialRing(R, 'y')
        y = P.gen()
        
        B = FactorBasis(m,q)
    
        #Explaining the factor basis
        if verbose:
            print("Attempting to compute the order of ", g, "modulo", n)
        numPrimes = len(B)
        if verbose:
            print("The number of irreducible polynomials in the factor base:", numPrimes)
            print("The factor base is given by:",B)
        
        # Set the number of relations to find
        relsDesired = numPrimes + rels  # the number of relations to find
        if verbose:
            print("We are looking for ", relsDesired, "relations.")
        vectors = []  # to store the relations we find
    
        # Main relation finding loop
        searchCount = 0 # number of smoothness tests run
        relsFound = 0 # number of relations found
        A = []
        maxlen = q^(n.degree())-1
        while(relsFound < relsDesired ):
            while(len(A)<maxlen):
                x = randint(1,maxlen)
                if x not in A:
                    A.append(x)
                    residue = d(g,n,x)
                    C = Division(residue,B,q)[0]
                    vec = vector([C[i] for i in range(0,numPrimes)])
                    remainder = Division(residue,B,q)[1]
                    if remainder == 1:
                        if verbose:
                            print("found a relation:", g, "^", x, "is", d(g,n,x))
                        if texify:
                            print(g,"^{",x,"}&=",latex(d(g,n,x)),",\\\\")
                        relsFound += 1
                        vectors.append([vec,x])
                    searchCount +=1
                    
            if len(A)== maxlen:
                relsDesired = relsFound
        if verbose:
            print("I searched through ", searchCount, "powers of ", g)
             #Linear algebra phase
        print("vectors", vectors)
        relMatrix = matrix([vectors[i][0] for i in range(relsDesired)]).transpose()
        if verbose:
            print("The relation matrix is (cols are relations):")
            print(relMatrix)
        if texify:
            print(latex(relMatrix))
        kernel = relMatrix.right_kernel()
        kernel= kernel.basis()
    
        if verbose:
            print("The right kernel has basis:")
            print(kernel) 
        if texify:
            print(latex(matrix(kernel)))
        alphaVals = []
        print("xs?",[vectors[i][1] for i in range(relsDesired)])
        for basisVec in kernel:
            alpha = sum([basisVec[i]*vectors[i][1] for i in range(relsDesired)])
            alphaVals.append(alpha)
        if verbose:
            print("The corresponding sums of the x's are:")
            print(alphaVals)
        if texify:
            print(latex(alphaVals))
        
        #GCD phase
        finalGCD = 0
        for alpha in alphaVals:
            finalGCD = gcd(finalGCD,alpha)                                      
            #The idea here is gcd(0,alpha_1)=alpha_1, and then we use that gcd(a,b,c)=gcd(gcd(a,b),c)
        
        #Report and return
        if verbose:
            print("Their gcd is:", finalGCD)
            print("******* Relation to reality? ******")
            print("Sage's factorization of", g, "is", factor(g))
            if d(g,n,finalGCD) == 1:
                print("Success! so", finalGCD, "is indeed a multiple of the gcd")
            else:
                print("Failure.. so", finalGCD, "is not a multiple of the gcd")
                
        if alphas:
            return(finalGCD,[a/finalGCD for a in alphaVals]) #Note that if we take LCM of this vector, we get 24.
        return finalGCD
    else: 
        print("choose another g mod n, (where g,n are coprime)")
get_order(y,y^2-1,101,80,rels=10,verbose=True,alphas = True, texify=True)

#Note that ((y^2+2y)mod(y^2+2))^2 mod (y^2+2) is indeed equal to y+2, in F3

KeyboardInterrupt: 